In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline
!pip install -qqq greykite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.1 MB/s et

In [3]:
# !pip install -qqq pandas
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/cameroon.csv')

df['Year-Month'] = df['Year-Month'].astype('datetime64[ns]')
df.rename(columns = {'Year-Month': 'ts', 'Cases': 'y'}, inplace = True)
df

,ts,y
0,2011-01-01,58
1,2011-02-01,147
2,2011-03-01,186
3,2011-04-01,105
4,2011-05-01,114
...,...,...
154,2023-11-01,16
155,2023-12-01,25
156,2024-01-01,73
157,2024-02-01,113


In [4]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [5]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

model_components = ModelComponentsParam(
    seasonality={
        "yearly_seasonality": False,      #12
        "quarterly_seasonality":False,
        "monthly_seasonality":False,
        "weekly_seasonality": False,
        "daily_seasonality": False,
        "auto_seasonality":False
    },
    growth={
        "growth_term": None  #linear, quadratic, sqrt, None
    },
    events={
        "holidays_to_model_separately": None  # "auto",["New Year's Day", "Thanksgiving"],None]
    },
   changepoints={                                 #"dict" or None
        "changepoints_dict": {
            "method": "auto",  # 'auto' to automatically detect changepoints
            "regularization_strength": 0.5,  # Regularization strength to control number of changepoints
            "potential_changepoint_n": 1,  # Number of potential changepoints to consider
            "no_changepoint_proportion_from_end": 0.1  # Proportion of data at the end where no changepoint is allowed
        }
    },
    autoregression={
        "autoreg_dict":"auto"   #"auto",None
    },
    uncertainty={
        "uncertainty_dict":  "auto"  #`dict` or "auto" or None

    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "quantile_regression"    #ridge, lasso, elastic_net, linear, quantile_regression
        }
    }

)

In [6]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=12,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata,
         model_components_param=model_components

    )
)

# Specifies dataset information
#metadata = MetadataParam(
    # time_col="ts",  # name of the time column
     #value_col="y",  # name of the value column
     #freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 #)

#forecaster = Forecaster()
#result = forecaster.run_forecast_config(
     #df=df,
     #config=ForecastConfig(
        # model_template=ModelTemplateEnum.SILVERKITE.name,
        # forecast_horizon=12,  # forecasts 100 steps ahead
        # coverage=0.95,  # 95% prediction intervals
       # 3 metadata_param=metadata
    #)
#)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [7]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

In [8]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [9]:
forecast_testt = result.backtest.df_test.forecast
actual_testt = result.backtest.df_test.actual

In [10]:
forecast_testt

147     91.229610
148     58.155701
149     65.240556
150     95.075331
151     65.240556
152     65.240556
153     80.990549
154     60.989000
155     42.084800
156    113.979530
157     65.240556
158     65.240556
Name: forecast, dtype: float64

In [11]:
actual_testt

147    1205
148     917
149     448
150      98
151      16
152      16
153      31
154      16
155      25
156      73
157     113
158       9
Name: actual, dtype: int64

In [18]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(actual_testt,forecast_testt))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actual_testt,forecast_testt)
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(actual_testt,forecast_testt)
#print("Mean Squared Logarithmic Error (MSLE):", msle)

def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(actual_testt, forecast_testt)

print('RMSE: %.2f' % error)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 422.43
MAPE: 177.01%
MAE:  226.15
MSLE: 2.36


In [19]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.121541,0.114726
R2,-0.092129,-0.196202
MSE,41608.975683,178449.905346
RMSE,203.982783,422.433315
MAE,96.630032,226.151983
MedAE,43.759444,49.240556
MAPE,143.025294,177.007708
MedAPE,69.055294,93.043563
sMAPE,40.937994,52.026423
Q80,67.446738,165.533309


In [21]:
# Access forecasted values
forecast_values = result.forecast.df

# Access forecasted components (e.g., trend, seasonality)
forecast_components = result.backtest.df_test
forecast_asd = result.forecast.df_test

# Print or display these results as needed
print("Forecast Values:")
print(forecast_components)


Forecast Values:
            ts  actual    forecast  forecast_lower  forecast_upper
147 2023-04-01    1205   91.229610     -289.826875      472.286095
148 2023-05-01     917   58.155701     -322.900784      439.212186
149 2023-06-01     448   65.240556     -315.815929      446.297040
150 2023-07-01      98   95.075331     -285.981154      476.131816
151 2023-08-01      16   65.240556     -315.815929      446.297040
152 2023-09-01      16   65.240556     -315.815929      446.297040
153 2023-10-01      31   80.990549     -300.065936      462.047033
154 2023-11-01      16   60.989000     -320.067485      442.045484
155 2023-12-01      25   42.084800     -338.971684      423.141285
156 2024-01-01      73  113.979530     -267.076954      495.036015
157 2024-02-01     113   65.240556     -315.815929      446.297040
158 2024-03-01       9   65.240556     -315.815929      446.297040
